In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from huggingface_hub import login, HfApi

In [ ]:
login()

In [ ]:
# Load CSV files using pandas
train_df = pd.read_csv('model_development\processed_data\train.csv')
test_df = pd.read_csv('model_development\processed_data\test.csv')

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

train_encodings = tokenize_texts(train_df['text'])
test_encodings = tokenize_texts(test_df['text'])

train_labels = torch.tensor(train_df['label'].values)
test_labels = torch.tensor(test_df['label'].values)

class ReviewDataset():
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

train_dataset = ReviewDataset(train_encodings, train_labels)
eval_dataset = ReviewDataset(test_encodings, test_labels)

In [32]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

model_configs = [
    {
        "model_name": "bert-base-uncased",
        "repo_name": "fake-review-detector-bert-base-uncased"
    },
    {
        "model_name": "roberta-base",
        "repo_name": "fake-review-detector-roberta-base"
    },
    {
        "model_name": "google/electra-base-discriminator",
        "repo_name": "fake-review-detector-google"
    }
]


# Store all results
results = []

# Loop through each model
for config in model_configs:
    model_name = config["model_name"]
    repo_name = config["repo_name"]
    hub_model_id = f"jesmine0820/{repo_name}"

    print(f"\n----------- Training with: {model_name}")

    # Create repo on HF Hub
    api = HfApi()
    api.create_repo(repo_id=repo_name, repo_type="model", exist_ok=True)

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Define training args
    training_args = TrainingArguments(
        output_dir=f"./results/{repo_name.replace('/', '_')}",
        do_eval=True,
        eval_steps=1000,
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=2,
        weight_decay=0.01,
        push_to_hub=True,
        hub_model_id=hub_model_id,
        hub_strategy="every_save",
        logging_strategy="no",
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate
    trainer.train()
    eval_metrics = trainer.evaluate()
    trainer.push_to_hub()

    # Add model info to metrics
    eval_metrics["model_name"] = model_name
    eval_metrics["repo_name"] = repo_name
    results.append(eval_metrics)

# Convert all results to DataFrame
df_results = pd.DataFrame(results)
print("\n >> Final Evaluation Results:\n")
print(df_results)


----------- Training with: bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


Uploading...:   0%|          | 0.00/438M [00:00<?, ?B/s]


----------- Training with: roberta-base


...


----------- Training with: google/electra-base-discriminator


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


Uploading...:   0%|          | 0.00/438M [00:00<?, ?B/s]


 >> Final Evaluation Results:

   eval_loss  eval_accuracy   eval_f1  eval_precision  eval_recall  \
0   0.190846         0.9296  0.929568        0.930282       0.9296   
1   0.310962         0.8756  0.875516        0.876512       0.8756   
2   0.377819         0.8670  0.865115        0.888206       0.8670   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  \
0       32.1055                  155.737                  4.890    2.0   
1       31.4704                  158.879                  4.989    2.0   
2       33.8359                  147.772                  4.640    2.0   

                          model_name                               repo_name  
0                  bert-base-uncased  fake-review-detector-bert-base-uncased  
1                       roberta-base       fake-review-detector-roberta-base  
2  google/electra-base-discriminator             fake-review-detector-google  


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model
model_id = "jesmine0820/fake-review-detector-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.eval()

# Label mapping
id2label = {0: "true", 1: "fake"}

# Sample test reviews
texts = [
    # True reviews
    "I received my order yesterday and everything works perfectly. Highly recommended.",
    "The phone arrived earlier than expected, and the packaging was great. Would buy again!",
    "Customer service was responsive and helped me resolve an issue within minutes.",
    "I've been using this laptop for two weeks, and performance is solid for the price.",

    # Fake reviews
    "Best product ever! Life changing! I will never use anything else again!",
    "Amazing amazing amazing. Five stars five stars five stars!",
    "This is the most wonderful thing I’ve ever purchased. 100% satisfied.",
    "Good product. Fast delivery. Good product. Fast delivery.",

    # Edge cases
    "It’s okay, but not as good as expected.",
    "Worked for a few days, then stopped. I might return it.",
    "Not sure if it's authentic, but it looks fine to me."
]

# Predict and print results
for i, text in enumerate(texts, 1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        predicted_class_id = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted_class_id].item()
        predicted_label = id2label.get(predicted_class_id, str(predicted_class_id))

    print(f"\nExample {i}")
    print(f"Text: {text}")
    print(f"Predicted Label: {predicted_label}")
    print(f"Confidence: {confidence:.4f}")
    print(f"Probabilities: {probs.tolist()[0]}")


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]